In [ ]:
!pip install streamlit pyngrok langchain langchain-community sentence-transformers chromadb

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:0

In [ ]:
from pyngrok import ngrok

# Replace YOUR_NGROK_AUTHTOKEN with your actual token
ngrok.set_auth_token("2vfd87yTSaW5YZS33l4ZGwErnzw_4k5QNjwrBLx7imx9eT7SE")

In [ ]:
%%writefile app.py
import os
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document

# ----------------- Streamlit UI Setup -----------------
st.set_page_config(page_title="Machine Learning Chatbot 🧠", page_icon="🤖")

st.title("🧠 Machine Learning Chatbot (Lecture Notes R17A0534)")
st.markdown("Ask anything about Machine Learning — concepts, algorithms, or examples!")

with st.sidebar:
    st.header("🔑 Configuration")
    openrouter_api_key = st.text_input("Enter your OpenRouter API Key", type="password")

# ----------------- Load PDF Automatically -----------------
PDF_PATH = "/content/drive/MyDrive/MACHINE LEARNING(R17A0534).pdf"

if not os.path.exists(PDF_PATH):
    st.error("❌ The Machine Learning PDF file was not found. Please place 'MACHINE LEARNING(R17A0534).pdf' in the same folder as app.py.")
else:
    if openrouter_api_key:
        try:
            with st.spinner("📚 Processing Machine Learning notes..."):
                # --- Safe UTF-8 PDF loading ---
                loader = PyPDFLoader(PDF_PATH)
                pages = loader.load_and_split()

                # Convert each page safely to UTF-8 text
                for i in range(len(pages)):
                    pages[i].page_content = pages[i].page_content.encode("utf-8", errors="ignore").decode("utf-8", errors="ignore")

                splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
                docs = splitter.split_documents(pages)
                documents = [Document(page_content=doc.page_content) for doc in docs]

                embeddings = HuggingFaceEmbeddings(
                    model_name="sentence-transformers/all-MiniLM-L6-v2",
                    model_kwargs={'device': 'cpu'}
                )
                vector_db = Chroma.from_documents(documents, embedding=embeddings)

                memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

                llm = ChatOpenAI(
                    model="openai/gpt-3.5-turbo",
                    temperature=0.2,
                    openai_api_base="https://openrouter.ai/api/v1",
                    max_tokens=500,
                    openai_api_key=openrouter_api_key
                )

                qa_chain = ConversationalRetrievalChain.from_llm(
                    llm=llm,
                    retriever=vector_db.as_retriever(),
                    memory=memory
                )

                st.success("✅ Machine Learning Chatbot ready! Start chatting below.")

                if "chat_history" not in st.session_state:
                    st.session_state.chat_history = []

                user_input = st.chat_input("Ask a question about Machine Learning...")

                if user_input:
                    with st.spinner("🤖 Thinking..."):
                        response = qa_chain({"question": user_input})
                        answer = response["answer"]
                        # Encode response safely
                        answer = answer.encode("utf-8", errors="ignore").decode("utf-8", errors="ignore")
                        st.session_state.chat_history.append(("user", user_input))
                        st.session_state.chat_history.append(("bot", answer))

                for role, msg in st.session_state.chat_history:
                    st.chat_message("user" if role == "user" else "assistant").write(msg)

        except Exception as e:
            st.error(f"❌ Error: {e}")
    else:
        st.warning("Please enter your OpenRouter API key in the sidebar.")


Overwriting app.py


In [ ]:
!pkill -f ngrok


In [ ]:

# ----------------- Ngrok Tunnel Setup -----------------
from pyngrok import ngrok
import threading, time

def run_streamlit():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=run_streamlit)
thread.start()
time.sleep(5)

public_url = ngrok.connect(8501)
print("🌐  Open your Machine Learning Chatbot at:", public_url)

🌐  Open your Machine Learning Chatbot at: NgrokTunnel: "https://fbf2c1a12d2a.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install pypdf
